In [ ]:
!pip install faiss-cpu  

In [ ]:
import pandas as pd
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer

# Load SBERT model
model = SentenceTransformer("sentence-transformers/paraphrase-mpnet-base-v2")

# Load dataset
df = pd.read_csv("ipc_sections.csv")  # Ensure correct file path

# Extract relevant columns
descriptions = df['Description'].tolist()
sections = df['Section'].tolist()
offenses = df['Offense'].tolist()
punishments = df['Punishment'].tolist()

# Convert descriptions into embeddings
embeddings = model.encode(descriptions, convert_to_numpy=True)

# Initialize FAISS index
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)  # Store embeddings in FAISS

# Function to find the most relevant IPC section
def find_ipc_section(case_description, k=3):
    query_embedding = model.encode([case_description], convert_to_numpy=True)
    D, I = index.search(query_embedding, k)  # Get top-k matches

    results = []
    for i in range(k):
        sec_num = sections[I[0][i]]
        sec_desc = descriptions[I[0][i]]
        offense = offenses[I[0][i]]
        punishment = punishments[I[0][i]]
        results.append((sec_num, offense, punishment, sec_desc, D[0][i]))

    return results

query_text = "murder"
matches = find_ipc_section(query_text, k=5)


for sec, offense, punishment, desc, dist in matches:
    print(f"IPC Section: {sec}\nOffense: {offense}\nPunishment: {punishment}\nDescription: {desc}\nDistance: {dist}\n---")


import pickle

data = {
    "sections": sections,
    "offenses": offenses,
    "punishments": punishments,
    "descriptions": descriptions,
}

with open("ipc_data.pkl", "wb") as f:
    pickle.dump(data, f)

    
import faiss
import pickle
from sentence_transformers import SentenceTransformer
from google.colab import files
import shutil

shutil.make_archive("sbert_model", 'zip', "sbert_model")

files.download("sbert_model.zip")  
files.download("faiss_index.faiss")  
files.download("ipc_data.pkl") 